## train 데이타와 일반대화 데이타를 merge하여 하나의 파일로 만든다.

0. 파일 
  - original_train_data : train_no_dup.csv
  - 일반대화 데이타 : conversation_normal.csv
1. train 데이타 내용 확인
2. 중복 데이타와 null 데이타 확인
3. original train과 일반 대화 데이타 merge
4. train 데이타 통계 분석 시각화 
5. merged train 데이타 csv 파일 저장

### 1. train 데이타 내용 확인

In [23]:
import os
import pandas as pd
import tensorflow
import numpy as np

In [24]:
orig_train_path ="../data/raw_csv/train_no_dup.csv"
orig_train = pd.read_csv(orig_train_path)
orig_train.head()

,idx,class,conversation
0,3748,직장 내 괴롭힘 대화,지은씨 커피 좀 사와요.\n그건 싫어요.\n사오라면 사와야지 인턴주제.\n그러니까요...
1,3149,직장 내 괴롭힘 대화,자 점심식사들 합시다\n 와 점심시간이다. 과장님 밥 어디서 먹죠 오늘은?\n 대리...
2,1866,협박 대화,어이 이쁜이 시간이 지났는데 약속을 안지키네\n네.엄마가 아프셔서.\n그래 엄마가 ...
3,1493,기타 괴롭힘 대화,우리 귀여운 철수야 여기 돈 줄테니까 핫도그랑 콜라좀 사다줄래? 남은 돈은 너 가져...
4,366,직장 내 괴롭힘 대화,어이 김대리\n예 이부장님\n오늘 하라고 했던 서류 정리 다 끝냈어?\n아.그게 아...


In [25]:
class_counts = orig_train['class'].value_counts()

print(class_counts)

orig_train.shape

기타 괴롭힘 대화      1011
갈취 대화           973
직장 내 괴롭힘 대화     970
협박 대화           892
Name: class, dtype: int64


(3846, 3)

In [26]:
# 일반 대화
normal_data_path ="../data/raw_csv/conversation_normal.csv"
normal_data = pd.read_csv(normal_data_path)
normal_data.head()

,idx,class,conversation
0,4001,일반 대화,"집에서 자주 만드는 음식 있어?\n응, 김치찌개 자주 만들어.\n오, 김치찌개! 어..."
1,4002,일반 대화,"너는 어떤 장르 좋아해?\n나는 록 음악을 좋아해.\n오, 록! 그럼 Queen 좋..."
2,4003,일반 대화,"오늘 점심 뭐 먹을까요?\n저는 김밥 먹고 싶은데, 다른 사람들은 어떻게 생각해요?..."
3,4004,일반 대화,과제 다 했어?\n아직이야. 너무 많아.\n나도 겨우 끝냈어.\n우리 같이 도서관 갈래?
4,4005,일반 대화,"이 장난감은 여러 가지 색상 중에서 선택할 수 있나요?\n네, 다양한 색상이 있어서..."


In [27]:
normal_data.shape

(932, 3)

### 2. 일반 대화 중복 데이타와 null 데이타 확인

In [28]:
print('normal_data의 전체 샘플수 :', (len(normal_data)))
print('normal_data의 conversation 열에서 중복을 배제한 유일한 샘플의 수 :', normal_data['conversation'].nunique())
print('normal_data의 중복제거 샘플수 :',(len(normal_data) - normal_data['conversation'].nunique()) )
print(normal_data.isnull().sum())

normal_data의 전체 샘플수 : 932
normal_data의 conversation 열에서 중복을 배제한 유일한 샘플의 수 : 791
normal_data의 중복제거 샘플수 : 141
idx             0
class           0
conversation    0
dtype: int64


In [29]:
normal_data.drop_duplicates(subset = ['conversation'], inplace=True)
print('normal_data의 중복 제거한 전체 샘플수 :', (len(normal_data)))

normal_data의 중복 제거한 전체 샘플수 : 791


### 3. Original Train과 일반대화 데이타 merge

In [30]:
train_data = pd.concat([orig_train, normal_data], ignore_index=True)


In [31]:
train_data = train_data.sample(frac=1).reset_index(drop=True)

In [32]:
class_counts = train_data['class'].value_counts()

print(class_counts)

기타 괴롭힘 대화      1011
갈취 대화           973
직장 내 괴롭힘 대화     970
협박 대화           892
일반 대화           791
Name: class, dtype: int64


In [33]:
train_data[train_data['class']=='일반 대화'][:3]

,idx,class,conversation
3,4999,일반 대화,"저녁 식사 뭐 먹을래?\n다음에 같이 가자. 내 말은, 요즘 교통 관련해서 뭐 재밌..."
12,4987,일반 대화,오랜만이다 잘 지냈어?\n커피 한잔할래?\n점심 뭐 먹었어?\n다음에 같이 가자.\...
22,4111,일반 대화,"오늘 운동했어?\n응, 아침에 헬스 다녀왔어.\n부지런하다! 무슨 운동했어?\n웨이..."


#### 4. merged_train 데이타 csv 파일로 저장

- ../data/raw_csv/merged_train.csv

In [38]:
train_data.to_csv('../data/raw_csv/merged_train.csv', index=False, encoding='utf-8-sig')  # index=False는 인덱스를 저장하지 않도록 함

In [39]:
train_data_path ="../data/raw_csv/merged_train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,1001,기타 괴롭힘 대화,얘들아. 선생님이 다음 수업은 실습실에서 한데.\n 야 어제 그거 개웃기지 않았냐?...
1,1252,직장 내 괴롭힘 대화,그 건설프로젝트 관련한 실험건 다 끝내서 보고서 작성까지 완료 하였습니다.\n아 그...
2,834,기타 괴롭힘 대화,너 치마 샀어?\n응응 이쁘지\n응 치마는 이쁘긴한데 \n뭐랄까 종아리 알 때문에 ...
3,4999,일반 대화,"저녁 식사 뭐 먹을래?\n다음에 같이 가자. 내 말은, 요즘 교통 관련해서 뭐 재밌..."
4,1852,갈취 대화,저기 학생 잠시만\n네\n미안한데 내가 지갑을 잃어버려서 그런데 집에갈 차비가없어서...


In [40]:
train_data.shape

(4637, 3)